# Getting started

In [3]:
import sweepystats as sw
import numpy as np

def random_symmetric_matrix(n, eigmin=float('-inf')):
    """
    Simulates an column-major double-precision
    n*n symmetric matrix with minimum eigenvalue `eigmin`
    """
    A = np.random.rand(n, n)
    A = 0.5 * (A + A.T)
    # force eigenvalues to be >= eigmin
    if eigmin > float('-inf'):
        eval, evec = np.linalg.eig(A)
        eval[np.where(eval < 0)[0]] = eigmin
        A = evec * np.diag(eval) * evec.T
    # convert to column major
    return np.array(A, order='F')

## The `SweepMatrix` class

`SweepMatrix` is a thin wrapper over numpy `darray`s. The input must be a symmetric matrix. We convert all matrices into double-precision numpy arrays with column-major order. **No data is copied** if the input is already in this format. The latter requirement is because the sweeping operation is blessed by level-3 BLAS, which we call internally. 

In [22]:
# intantiate a SweepMatrix from symmetric input
A_numpy = np.array([[1., 2, 3],
                    [2, 5, 6],
                    [3, 6, 9]], order='F') # order = 'F' implies data is stored in column-major format
A = sw.SweepMatrix(A_numpy)
A

SweepMatrix(array([[1., 2., 3.],
       [2., 5., 6.],
       [3., 6., 9.]]))

In [23]:
# modifying entries of A also changes the original
A[0, 0] = 10
A_numpy

array([[10.,  2.,  3.],
       [ 2.,  5.,  6.],
       [ 3.,  6.,  9.]])

A `SweepMatrix` can be swept forward and backwards:

In [24]:
A.sweep(verbose=False) # forward sweep through all diagonal entries of A
A

SweepMatrix(array([[-1.11111111e-01, -4.85722573e-17,  3.70370370e-02],
       [-4.85722573e-17, -1.00000000e+00,  6.66666667e-01],
       [ 3.70370370e-02,  6.66666667e-01, -5.67901235e-01]]))

In [25]:
A.sweep(inv=True, verbose=False) # inverse sweep recovers the original data
A

SweepMatrix(array([[10.,  2.,  3.],
       [ 2.,  5.,  6.],
       [ 3.,  6.,  9.]]))

We can also sweep a on the `k`th diagonal element

In [26]:
A.sweep_k(1) # sweep the kth diagonal
A

SweepMatrix(array([[ 9.2,  0.4,  0.6],
       [ 0.4, -0.2,  1.2],
       [ 0.6,  1.2,  1.8]]))

In [27]:
A.sweep_k(1, inv=True) # unweep the kth diagonal
A

SweepMatrix(array([[10.,  2.,  3.],
       [ 2.,  5.,  6.],
       [ 3.,  6.,  9.]]))

Because the sweep operation divides the `k`th row/column of A by `Akk`, the diagonal entry **cannot be 0** (exactly or numerically). 

## Numerical linear algebra

The following operations are supported **in-place** and **allocation-free**
+ Matrix inversion in $n^3$ flops. Recall that inversion by Cholesky costs $(5/3)n^3$ flops and needs to allocate a matrix of same size.
+ Computation of determinant
+ Check (strict) positive definiteness

Matrix inverse:

In [10]:
# A.sweep() converts A.A into -inv(A.A)
data = random_symmetric_matrix(100)
Ainv = np.linalg.inv(data)
A = sw.SweepMatrix(data)
A.sweep()
np.allclose(A.A, -Ainv) # check answer is correct

100%|██████████████████████████████████████████████| 100/100 [00:00<00:00, 3924.02it/s]


True

Determinants:

In [9]:
# Determinants is computed by A.det(). Original matrix is untouched by default. 
data = random_symmetric_matrix(100)
Adet = np.linalg.det(data)
A = sw.SweepMatrix(data)
np.allclose(A.det(restore=True), Adet) # check answer is correct

100%|████████████████████████████████████████████| 100/100 [00:00<00:00, 124867.64it/s]


True

`det` can return exactly 0 if matrix have 0 as eigenvalu

In [18]:
# det can return exactly 0 if matrix have 0 as eigenvalue
data = random_symmetric_matrix(100, eigmin=0.0)
data_original = data.copy()
Adet = np.linalg.det(data)
A = sw.SweepMatrix(data)
A.det(verbose=False) == 0.0

True

Check positive definiteness:

In [15]:
A = sw.SweepMatrix(random_symmetric_matrix(100, eigmin=0.0001)) # this is PD
A.isposdef(verbose=False)

True

In [16]:
A = sw.SweepMatrix(random_symmetric_matrix(100, eigmin=-10)) # this has negative eigenvalues, should return false
A.isposdef(verbose=False)

False

In [17]:
A = sw.SweepMatrix(random_symmetric_matrix(100, eigmin=0.0)) # this is PSD, should return false
A.isposdef(verbose=False)

False